In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Multi-vehicle VRP with time windows (Gurobi)

– Reads Data.tsv exactly like your single-truck version
– Solves for m identical trucks
– Prints each truck’s route with HH:MM arrival / departure
– Flags unloading stops whenever cumulative volume would breach capacity
– Plots the solution (green lines) and client locations (blue dots)

Dependencies:
    pip install pandas numpy matplotlib gurobipy
"""

import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import math
import matplotlib.pyplot as plt

# ============================================================================
# USER SETTINGS
# ============================================================================
m            = 15          # number of vehicles (changed from 15 to 20)
Capacity     = 1000       # litres
AvgVelocity  = 30         # km/h
WorkStart    = 9 * 3600   # seconds  (09:00)

# ============================================================================
# DATA -----------------------------------------------------------------------
# ============================================================================
data0 = pd.read_csv("Data_200.tsv", delimiter="\t")

# coordinates (simple 5× scale to make small DD coords look like km)
xc = list(data0['lon'] * 5)
yc = list(data0['lat'] * 5)
xDepot = np.mean(xc)
yDepot = np.mean(yc)
xc += [xDepot, xDepot]     # two virtual depots (start & end)
yc += [yDepot, yDepot]

AllNodes = list(range(len(xc)))
CusNodes = AllNodes[:-2]                # drop the two depot indices
SDep, EDep = len(AllNodes) - 2, len(AllNodes) - 1

Edges     = [(i, j) for i in AllNodes for j in AllNodes if i != j]
Distance  = {(i, j): np.hypot(xc[i]-xc[j], yc[i]-yc[j]) for i, j in Edges}
for i in AllNodes:                      # start-depot arcs cost = 0
    Distance[SDep, i] = 0
TravelTime = {(i, j): Distance[i, j] / (AvgVelocity/3600) for i, j in Edges}

PickAmount   = {i: data0['size'][i] * data0['quantity'][i] for i in CusNodes}
ServiceTime  = {i: data0['service_time'][i]                  for i in CusNodes}

# convert 'HH' or 'H:' style strings to hours float
time1 = [float(ts[0]) if ts[1] == ':' else float(ts[:2]) for ts in data0['time_start']]
time2 = [26.0 if pd.isna(te) else
         (float(te[0]) if te[1] == ':' else float(te[:2])) for te in data0['time_end']]
StartTime = {i: time1[i]*3600 for i in CusNodes}
EndTime   = {i: time2[i]*3600 for i in CusNodes}

# ============================================================================
# MODEL ----------------------------------------------------------------------
# ============================================================================
mdl = gp.Model("Seenons_multi")

K = range(m)                                          # vehicle set

x  = mdl.addVars(Edges, K, vtype=GRB.BINARY,     name="x")     # routing
Ar = mdl.addVars(CusNodes, K, vtype=GRB.CONTINUOUS, name="arr")
De = mdl.addVars(CusNodes, K, vtype=GRB.CONTINUOUS, name="dep")

# Objective – minimise total driven distance
mdl.setObjective(gp.quicksum(x[i, j, k] * Distance[i, j]
                             for i, j in Edges for k in K),
                 GRB.MINIMIZE)

# 1. Every customer visited exactly once
mdl.addConstrs(
    gp.quicksum(x[i, j, k] for i in AllNodes if i != j for k in K) == 1
    for j in CusNodes)

# 2. Flow conservation per vehicle
for k in K:
    mdl.addConstr(gp.quicksum(x[SDep, j, k] for j in AllNodes if j != SDep) == 1)
    mdl.addConstr(gp.quicksum(x[i, EDep, k] for i in AllNodes if i != EDep) == 1)
    mdl.addConstrs(
        gp.quicksum(x[i, j, k] for i in AllNodes if i != j) ==
        gp.quicksum(x[j, h, k] for h in AllNodes if h != j)
        for j in CusNodes)

# 3. Time-window & sequencing logic
M = 90000
mdl.addConstrs(De[i, k] >= Ar[i, k] + ServiceTime[i]
               for i in CusNodes for k in K)
mdl.addConstrs(Ar[i, k] >= StartTime[i]
               for i in CusNodes for k in K)
mdl.addConstrs(Ar[i, k] <= EndTime[i]
               for i in CusNodes for k in K)

mdl.addConstrs(
    De[i, k] + TravelTime[i, j] - Ar[j, k] <= (1 - x[i, j, k]) * M
    for i in CusNodes for j in CusNodes if i != j for k in K)

mdl.addConstrs(
    (x[SDep, j, k] == 1) >> (Ar[j, k] == WorkStart)
    for j in CusNodes for k in K)

# ============================================================================
# SOLVE ----------------------------------------------------------------------
# ============================================================================
#mdl.setParam(GRB.Param.TimeLimit, 20)  # 20-second limit
mdl.optimize()

# Print the objective value (total distance)
if mdl.SolCount > 0:
    print(f"\nTotal distance (objective): {mdl.ObjVal:.2f} km\n")
else:
    print("\nNo feasible solution found within time limit.\n")

# ============================================================================
# POST-PROCESSING ------------------------------------------------------------
# ============================================================================
def sec_to_hhmm(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    return f"{h:02d}:{m:02d}"

print("================  SOLUTION  ================\n")

for k in K:
    active = [(i, j) for i, j in Edges if x[i, j, k].X > 0.9]
    if not active:
        print(f"Vehicle {k}: not used.\n")
        continue

    route, sn = [], SDep
    while sn != EDep:
        for (i, j) in active:
            if i == sn:
                route.append((i, j)); sn = j; break
    seq = [t[1] for t in route[:-1]]

    unload = {i: "No" for i in seq}
    totload = 0
    for idx, node in enumerate(seq[:-1]):
        totload += PickAmount[node]
        if totload + PickAmount[seq[idx+1]] >= Capacity:
            unload[node] = "Yes"
            totload = 0

    print(f"Vehicle {k}")
    print("Node  Arr    Dep    Unload")
    for n in seq:
        arr = sec_to_hhmm(Ar[n, k].X)
        dep = sec_to_hhmm(De[n, k].X)
        print(f"{n:>4}  {arr}  {dep}   {unload[n]}")
    print()

    # plotting
    for (i, j) in active:
        if i not in (SDep, EDep) and j not in (SDep, EDep):
            plt.plot([xc[i], xc[j]], [yc[i], yc[j]], c='g', zorder=0)

plt.scatter(xc[:-2], yc[:-2], c='b', zorder=3)
plt.title(f"Solution with {m} vehicle(s)")  # now reflects 20 vehicles
plt.axis('equal')
plt.show()


using 200 data points

Set parameter Username
Set parameter LicenseID to value 2694539
Academic license - for non-commercial use only - expires 2026-08-08
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 609230 rows, 615030 columns and 3618030 nonzeros
Model fingerprint: 0xde004b4f
Model has 3000 simple general constraints
  3000 INDICATOR
Variable types: 6000 continuous, 609030 integer (609030 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+04]
  Objective range  [2e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+04]
  GenCon rhs range [3e+04, 3e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 114885 rows and 86385 columns
Presolve time: 4.61s
Presolved: 494345 rows, 528645 columns, 3036765 nonzeros
Variable types: 3000 continuous, 525645 integer (525645 binary)
Found heuristic solution: objective 7607.0228755
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.02s

Barrier performed 0 iterations in 6.61 seconds (17.45 work units)
Barrier solve interrupted - model solved by another algorithm

Concurrent spin time: 0.06s

Solved with dual simplex

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     601    4.7272722e+02   0.000000e+00   0.000000e+00      7s

Use crossover to convert LP symmetric solution to basic solution...

Root crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 7s

    1553 PPushes remaining with PInf 0.0000000e+00                 7s
       0 PPushes remaining with PInf 0.0000000e+00                 7s

  Push phase complete: Pinf 0.0000000e+00, Dinf 1.6954681e+01      7s


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4847    4.7272722e+02   0.000000e+00   1.695468e+01      7s
    6147    4.7272721e+02   0.000000e+00   9.804592e+00     10s
    6735    4.7272722e+02   0.000000e+00   0.000000e+00     11s

Root relaxation: objective 4.727272e+02, 6735 iterations, 4.74 seconds (20.42 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  472.72722    0  409 7607.02288  472.72722  93.8%     -   11s
H    0     0                    1897.9750816  472.72722  75.1%     -   12s
H    0     0                    1592.2043408  472.72722  70.3%     -   12s
H    0     0                    1577.2191919  472.72722  70.0%     -   12s
H    0     0                    1576.3596027  472.72722  70.0%     -   12s
H    0     0                    1554.6925703  472.72722  69.6%     -   12s
H    0     0                    1530.2674144  472.72722  69.1%     -   12s
H    0     0                    1522.7933435  472.72722  69.0%     -   12s
H    0     0                    1511.3380126  472.72722  68.7%     -   12s
H    0     0                    1502.0341934  472.72722  68.5%     -   12s
H    0     0                    1482.6100635  472.72722  68.1%     -   12s
H    0     0                    1465.8004952  472.72722  67.7%     -   12s
H    0     0                    1457.4110076  472.72722  67.6%     -   12s
H    0     0                    1455.5214626  472.72722  67.5%     -   12s
H    0     0                    1451.7887254  472.72722  67.4%     -   12s
H    0     0                    1435.2142266  472.72722  67.1%     -   14s
H    0     0                    1409.2497682  472.72722  66.5%     -   15s
H    0     0                    1393.9449956  472.72722  66.1%     -   15s
H    0     0                    1382.0841488  472.72722  65.8%     -   16s
H    0     0                    1357.1795228  472.72722  65.2%     -   17s
H    0     0                    1355.3770218  472.72722  65.1%     -   18s
H    0     0                    1347.8664970  472.72722  64.9%     -   18s
H    0     0                    1346.2466036  472.72722  64.9%     -   18s
H    0     0                    1339.6889334  472.72722  64.7%     -   18s
H    0     0                    1337.8225114  472.72722  64.7%     -   18s
H    0     0                    1303.8025829  472.72722  63.7%     -   18s
H    0     0                    1293.7095815  472.72722  63.5%     -   19s
H    0     0                    1287.0059584  472.72722  63.3%     -   19s
H    0     0                    1267.6308926  472.72722  62.7%     -   20s
     0     0  472.72722    0  464 1267.63089  472.72722  62.7%     -   23s
H    0     0                    1265.3579199  472.72722  62.6%     -   25s
H    0     0                    1258.3665815  472.72722  62.4%     -   31s
H    0     0                    1218.7850162  472.72722  61.2%     -   32s
H    0     0                    1206.4777854  472.72722  60.8%     -   33s
H    0     0                    1175.3014576  472.72722  59.8%     -   34s
H    0     0                    1163.0265719  472.72722  59.4%     -   34s
H    0     0                    1158.2525523  472.72722  59.2%     -   34s
H    0     0                    1154.5982078  472.72722  59.1%     -   36s
H    0     0                    1153.9923116  472.72722  59.0%     -   37s
H    0     0                    1152.2530096  472.72722  59.0%     -   37s
H    0     0                    1152.2330937  472.72722  59.0%     -   37s
H    0     0                    1152.1965208  472.72722  59.0%     -   37s
H    0     0                    1150.3689214  472.72722  58.9%     -   37s
     0     0  472.72722    0  519 1150.36892  472.72722  58.9%     -   38s
     0     0  472.72722    0  409 1150.36892  472.72722  58.9%     -   42s
     0     0  472.72722    0  455 1150.36892  472.72722  58.9%     -   44s
     0     0  472.72722    0  418 1150.36892  472.72722  58.9%     -   48s
     0     0  472.72722    0  463 1150.36892  472.72722  58.9%     -   50s
     0     0  472.72722    0  410 1150.36892  472.72722  58.9%     -   55s
     0     0  472.72722    0  436 1150.36892  472.72722  58.9%     -   56s
     0     0  472.81205    0  417 1150.36892  472.81205  58.9%     -   62s
     0     0  473.69099    0  451 1150.36892  473.69099  58.8%     -   69s
     0     0  482.91783    0  440 1150.36892  482.91783  58.0%     -   73s
     0     0  483.30814    0  434 1150.36892  483.30814  58.0%     -   74s
     0     0  483.30814    0  452 1150.36892  483.30814  58.0%     -   78s
     0     0  488.16095    0  583 1150.36892  488.16095  57.6%     -   85s
     0     0  493.98380    0  562 1150.36892  493.98380  57.1%     -   89s
     0     0  493.99334    0  527 1150.36892  493.99334  57.1%     -   91s
     0     0  493.99334    0  546 1150.36892  493.99334  57.1%     -   93s
     0     0  513.84822    0  551 1150.36892  513.84822  55.3%     -  102s
     0     0  514.38596    0  554 1150.36892  514.38596  55.3%     -  108s
     0     0  514.38596    0  547 1150.36892  514.38596  55.3%     -  111s
     0     0  522.33637    0  610 1150.36892  522.33637  54.6%     -  122s
H    0     0                    1144.4466874  522.33637  54.4%     -  126s
H    0     0                    1139.6718133  522.33637  54.2%     -  128s
H    0     0                    1134.7364004  522.33637  54.0%     -  129s
H    0     0                    1134.6506381  522.33637  54.0%     -  130s
H    0     0                    1120.7359184  522.33637  53.4%     -  132s
H    0     0                    1119.2766278  522.33637  53.3%     -  132s
H    0     0                    1105.9635761  522.33637  52.8%     -  138s
H    0     0                    1090.5372000  522.33637  52.1%     -  139s
H    0     0                    1086.2904881  522.33637  51.9%     -  144s
H    0     0                    1072.6540375  522.33637  51.3%     -  150s
H    0     0                    1058.1429049  522.33637  50.6%     -  150s
H    0     0                    1057.1962151  522.33637  50.6%     -  152s
     0     0  522.48033    0  575 1057.19622  522.48033  50.6%     -  160s
     0     0  522.48819    0  533 1057.19622  522.48819  50.6%     -  163s
     0     0  527.68903    0  569 1057.19622  527.68903  50.1%     -  170s
H    0     0                    1056.5522352  527.68903  50.1%     -  172s
H    0     0                    1056.4749691  527.68903  50.1%     -  172s
H    0     0                    1054.9697841  527.68903  50.0%     -  172s
     0     0  527.70962    0  529 1054.96978  527.70962  50.0%     -  173s
     0     0  527.70962    0  543 1054.96978  527.70962  50.0%     -  174s
     0     0  531.25857    0  596 1054.96978  531.25857  49.6%     -  184s
     0     0  531.80190    0  573 1054.96978  531.80190  49.6%     -  188s
     0     0  531.80190    0  542 1054.96978  531.80190  49.6%     -  189s
     0     0  532.73546    0  532 1054.96978  532.73546  49.5%     -  208s
H    0     0                    1053.5022809  532.73546  49.4%     -  209s
     0     0  532.77288    0  477 1053.50228  532.77288  49.4%     -  213s
     0     0  533.14763    0  471 1053.50228  533.14763  49.4%     -  216s
     0     0  533.14763    0  459 1053.50228  533.14763  49.4%     -  217s
     0     0  534.73245    0  547 1053.50228  534.73245  49.2%     -  227s
     0     0  534.73245    0  513 1053.50228  534.73245  49.2%     -  230s
     0     0  535.07113    0  484 1053.50228  535.07113  49.2%     -  235s
H    0     0                    1051.5849845  535.07113  49.1%     -  243s
H    0     0                    1012.7567406  535.07113  47.2%     -  245s
H    0     0                    1012.7423769  535.07113  47.2%     -  246s
H    0     0                    1008.6382857  535.07113  47.0%     -  247s
H    0     0                    1002.7430946  535.07113  46.6%     -  248s
H    0     0                     990.6857533  535.07113  46.0%     -  250s
H    0     0                     990.2647382  535.07113  46.0%     -  253s
H    0     0                     982.5189526  535.07113  45.5%     -  256s
H    0     0                     981.8056150  535.07113  45.5%     -  257s
H    0     0                     977.5540031  535.07113  45.3%     -  267s
H    0     0                     972.2501258  535.07113  45.0%     -  269s
H    0     0                     963.9545942  535.07113  44.5%     -  283s
H    0     0                     958.8916078  535.07113  44.2%     -  299s
H    0     0                     958.6517903  535.07113  44.2%     -  299s
H    0     0                     955.2250132  535.07113  44.0%     -  300s
H    0     0                     954.2769447  535.07113  43.9%     -  300s
H    0     0                     953.3819264  535.07113  43.9%     -  300s
H    0     0                     950.4827601  535.07113  43.7%     -  304s
H    0     0                     949.6867425  535.07113  43.7%     -  319s
H    0     0                     948.7267821  535.07113  43.6%     -  319s
H    0     0                     947.3114772  535.07113  43.5%     -  323s
H    0     0                     942.8670866  535.07113  43.3%     -  326s
H    0     0                     942.2235176  535.07113  43.2%     -  326s
H    0     0                     941.5989857  535.07113  43.2%     -  326s
H    0     0                     938.7892825  535.07113  43.0%     -  326s
     0     0  535.09820    0  515  938.78928  535.09820  43.0%     -  332s
     0     0  535.09820    0  498  938.78928  535.09820  43.0%     -  333s
     0     0  536.04325    0  587  938.78928  536.04325  42.9%     -  340s
     0     0  536.14553    0  560  938.78928  536.14553  42.9%     -  346s
     0     0  536.15587    0  518  938.78928  536.15587  42.9%     -  347s
     0     0  538.11440    0  577  938.78928  538.11440  42.7%     -  363s
     0     0  538.11440    0  524  938.78928  538.11440  42.7%     -  366s
     0     0  540.17049    0  538  938.78928  540.17049  42.5%     -  373s
     0     0  540.17049    0  583  938.78928  540.17049  42.5%     -  377s
     0     0  542.33070    0  566  938.78928  542.33070  42.2%     -  382s
     0     0  542.46754    0  496  938.78928  542.46754  42.2%     -  384s
     0     0  542.46754    0  492  938.78928  542.46754  42.2%     -  385s
     0     0  543.75470    0  589  938.78928  543.75470  42.1%     -  392s
     0     0  543.76677    0  664  938.78928  543.76677  42.1%     -  449s
     0     0  544.68089    0  537  938.78928  544.68089  42.0%     -  483s
     0     0  545.80957    0  628  938.78928  545.80957  41.9%     -  564s
     0     0  545.80957    0  603  938.78928  545.80957  41.9%     -  566s
     0     0  546.51175    0  541  938.78928  546.51175  41.8%     -  572s
H    0     0                     938.3404307  546.51175  41.8%     -  575s
H    0     0                     937.4381649  546.51175  41.7%     -  577s
H    0     0                     937.0962898  546.51175  41.7%     -  577s
H    0     0                     936.6421288  546.51175  41.7%     -  579s
H    0     0                     931.0698447  546.51175  41.3%     -  579s
     0     0  546.51175    0  516  931.06984  546.51175  41.3%     -  581s
     0     0  547.52076    0  617  931.06984  547.52076  41.2%     -  586s
     0     0  547.78486    0  610  931.06984  547.78486  41.2%     -  648s
     0     0  547.78486    0  552  931.06984  547.78486  41.2%     -  679s
     0     0  548.84164    0  659  931.06984  548.84164  41.1%     -  686s
H    0     0                     929.7512092  548.84164  41.0%     -  689s
H    0     0                     927.1504775  548.84164  40.8%     -  690s
H    0     0                     926.0271353  548.84164  40.7%     -  691s
H    0     0                     920.1754194  548.84164  40.4%     -  691s
H    0     0                     920.0263756  548.84164  40.3%     -  692s
H    0     0                     919.1514045  548.84164  40.3%     -  693s
H    0     0                     916.0840961  548.84164  40.1%     -  693s
     0     0  548.94024    0  619  916.08410  548.94024  40.1%     -  701s
     0     0  548.94024    0  629  916.08410  548.94024  40.1%     -  703s
     0     0  549.23270    0  546  916.08410  549.23270  40.0%     -  708s
     0     0  549.29916    0  529  916.08410  549.29916  40.0%     -  713s
     0     0  549.29916    0  531  916.08410  549.29916  40.0%     -  715s
     0     0  549.98727    0  564  916.08410  549.98727  40.0%     -  755s
H    0     0                     915.7319120  549.98727  39.9%     -  758s
     0     0  550.18626    0  639  915.73191  550.18626  39.9%     -  762s
     0     0  550.18626    0  584  915.73191  550.18626  39.9%     -  763s
     0     0  551.16503    0  618  915.73191  551.16503  39.8%     -  770s
     0     0  551.17414    0  646  915.73191  551.17414  39.8%     -  775s
     0     0  551.46158    0  703  915.73191  551.46158  39.8%     -  780s
H    0     0                     914.8925887  551.46158  39.7%     -  785s
H    0     0                     910.9807256  551.46158  39.5%     -  786s
     0     0  551.53367    0  542  910.98073  551.53367  39.5%     -  788s
     0     0  551.53367    0  521  910.98073  551.53367  39.5%     -  789s
     0     0  551.64605    0  522  910.98073  551.64605  39.4%     -  793s
     0     0  551.64605    0  540  910.98073  551.64605  39.4%     -  796s
     0     0  551.65686    0  698  910.98073  551.65686  39.4%     -  800s
     0     0  551.65686    0  436  910.98073  551.65686  39.4%     -  809s
H    0     0                     910.7820094  557.65288  38.8%     -  989s
H    0     0                     909.9103935  557.65288  38.7%     -  997s
H    0     0                     909.4770968  557.65288  38.7%     - 1003s
H    0     0                     903.9496210  557.65288  38.3%     - 1007s
H    0     0                     902.6753266  557.65288  38.2%     - 1017s
H    0     0                     900.9299109  557.65288  38.1%     - 1022s
     0     2  557.65288    0  403  900.92991  557.65288  38.1%     - 1028s
     1     3  557.65288    1  512  900.92991  557.65288  38.1%  8154 1035s
     2     6  557.65288    2  622  900.92991  557.65288  38.1%  6050 1049s
     5    12  557.65288    3  647  900.92991  557.65288  38.1%  4959 1058s
    11    22  557.65288    4  619  900.92991  557.65288  38.1%  3449 1080s
    21    36  557.65288    5  588  900.92991  557.65288  38.1%  3324 1096s
    35    86  557.65288    6  586  900.92991  557.65288  38.1%  2604 1114s
    85   114  557.65288   10  576  900.92991  557.65288  38.1%  1319 1124s
   113   137  557.65288   11  636  900.92991  557.65288  38.1%  1126 1135s
   136   157  557.65288   12  632  900.92991  557.65288  38.1%  1007 1144s
   156   187  557.65288   13  640  900.92991  557.65288  38.1%   974 1155s
   186   201  557.65288   16  597  900.92991  557.65288  38.1%   900 1167s
   200   216  557.65288   16  617  900.92991  557.65288  38.1%   891 1186s
H  203   216                     899.8704841  557.65288  38.0%   882 1186s
   215   243  557.65288   17  560  899.87048  557.65288  38.0%   863 1196s
   242   264  557.65288   18  561  899.87048  557.65288  38.0%   814 1206s
   264   290  557.65288   19  551  899.87048  557.65288  38.0%   793 1216s
   290   314  557.65288   20  569  899.87048  557.65288  38.0%   771 1226s
   314   337  557.65288   14  599  899.87048  557.65288  38.0%   740 1235s
   337   353  557.65288   22  599  899.87048  557.65288  38.0%   716 1245s
   353   377  557.65288   23  600  899.87048  557.65288  38.0%   712 1257s
   377   402  557.65288   25  582  899.87048  557.65288  38.0%   715 1271s
   402   443  557.65288   26  560  899.87048  557.65288  38.0%   719 1283s
   443   457  557.65288   27  549  899.87048  557.65288  38.0%   688 1296s
   457   488  557.65288   28  565  899.87048  557.65288  38.0%   692 1317s
   488   517  557.65288   29  580  899.87048  557.65288  38.0%   678 1329s
   517   531  557.65288   30  564  899.87048  557.65288  38.0%   669 1341s
H  518   531                     884.5436061  557.65288  37.0%   668 1341s
   531   567  557.65288   31  561  884.54361  557.65288  37.0%   668 1354s
   567   598  557.65288   36  562  884.54361  557.65288  37.0%   655 1368s
   599   646  560.61311   37  593  884.54361  557.65288  37.0%   660 1381s
   647   703  560.61311   38  597  884.54361  557.65288  37.0%   647 1395s
   704   763  561.32918   45  621  884.54361  557.65288  37.0%   632 1411s
   764   818  562.43892   56  584  884.54361  557.65288  37.0%   619 1427s
   819   869  564.95688   63  610  884.54361  557.65288  37.0%   611 1443s
   871   926  570.98563   71  556  884.54361  557.65288  37.0%   609 1460s
H  883   926                     883.9186050  557.65288  36.9%   607 1460s
H  886   926                     883.0755278  557.65288  36.9%   607 1460s
H  892   926                     882.4522124  557.65288  36.8%   607 1460s
   929   995  573.63597   73  518  882.45221  557.65288  36.8%   599 1478s
   998  1009  573.68352   80  509  882.45221  557.65288  36.8%   588 1517s
H 1007  1009                     877.7432976  557.65288  36.5%   587 1518s
  1012  1067  573.68352   81  508  877.74330  557.65288  36.5%   588 1537s
H 1070  1081                     873.9393969  557.65288  36.2%   593 1558s
H 1076  1081                     873.7397790  557.65288  36.2%   593 1558s
H 1077  1081                     868.8251625  557.65288  35.8%   593 1558s
  1084  1146  574.19933   89  542  868.82516  557.65288  35.8%   590 1576s
  1149  1235  575.25453   95  502  868.82516  557.65288  35.8%   589 1596s
  1238  1325  576.65598  103  507  868.82516  557.65288  35.8%   579 1617s
  1329  1428  590.57954  110  574  868.82516  557.65288  35.8%   569 1639s
  1432  1536  591.23971  113  554  868.82516  557.65288  35.8%   558 1660s
  1540  1654  594.69462  120  545  868.82516  557.65288  35.8%   546 1683s
  1662  1787  597.19165  130  465  868.82516  557.65288  35.8%   532 1708s
  1796  1924  598.34306  138  402  868.82516  557.65288  35.8%   516 1732s
  1934  2029  603.60376  155  467  868.82516  557.65288  35.8%   505 1756s
H 2039  2043                     866.8048822  557.65288  35.7%   504 1791s
H 2040  2043                     866.3405327  557.65288  35.6%   504 1791s
H 2045  2043                     866.3329778  557.65288  35.6%   503 1791s
H 2046  2043                     865.6352001  557.65288  35.6%   503 1791s
H 2047  2043                     864.5268631  557.65288  35.5%   503 1791s
H 2052  2043                     862.2980454  557.65288  35.3%   504 1791s
  2053  2185  611.93410  164  488  862.29805  557.65288  35.3%   505 1820s
  2196  2328  614.00732  172  480  862.29805  557.65288  35.3%   495 1849s
  2342  2468  615.29587  181  469  862.29805  557.65288  35.3%   487 1881s
  2484  2593  616.05380  189  513  862.29805  557.65288  35.3%   483 1916s
  2623  2744  618.06355  205  446  862.29805  557.65288  35.3%   480 1949s
  2775  2887  619.48601  222  471  862.29805  557.65288  35.3%   475 1984s
  2920  3022  641.86838  229  493  862.29805  557.65288  35.3%   473 2019s
  3056  3170  642.60256  241  511  862.29805  557.65288  35.3%   474 2053s
H 3066  3170                     861.3476937  557.65288  35.3%   474 2053s
H 3121  3170                     860.5484760  557.65288  35.2%   471 2053s
  3204  3348  658.41208  251  510  860.54848  557.65288  35.2%   473 2087s
  3382  3492  663.32700  263  496  860.54848  557.65288  35.2%   467 2128s
H 3526  3547                     856.5299929  557.65288  34.9%   471 2166s
  3589  3758  666.33797  279  443  856.52999  557.65288  34.9%   472 2207s
  3805  3985  683.44636  298  469  856.52999  557.65288  34.9%   466 2250s
  4032  4171  686.20531  311  423  856.52999  557.65288  34.9%   462 2292s
H 4218  4181                     852.5774653  557.65288  34.6%   463 2353s
H 4219  4181                     851.1321444  557.65288  34.5%   463 2353s
H 4220  4181                     850.5454900  557.65288  34.4%   464 2353s
H 4226  4172                     847.8023877  557.65288  34.2%   464 2353s
  4232  4362  688.35313  338  448  847.80239  557.65288  34.2%   464 2395s
  4423  4556  691.02379  369  429  847.80239  557.65288  34.2%   465 2443s
  4617  4794  696.08567  387  440  847.80239  557.65288  34.2%   469 2492s
H 4857  4805                     843.7872920  557.65288  33.9%   468 2555s
H 4863  4805                     842.9148339  557.65288  33.8%   468 2555s
H 4865  4804                     841.5037300  557.65288  33.7%   468 2555s
H 4867  4802                     838.2676467  557.65288  33.5%   468 2555s
H 4868  4802                     836.1768088  557.65288  33.3%   468 2555s
  4871  5041  696.97728  419  463  836.17681  557.65288  33.3%   468 2606s
  5111  5281  714.96148  447  366  836.17681  557.65288  33.3%   470 2659s
  5351  5503  725.22869  465  401  836.17681  557.65288  33.3%   472 2711s
  5577  5785  733.77468  468  429  836.17681  557.65288  33.3%   477 2769s
  5859  6062  738.49826  493  439  836.17681  557.65288  33.3%   477 2828s
  6137  6336  745.97987  513  380  836.17681  557.65288  33.3%   480 2888s
H 6173  6336                     835.5472359  557.65288  33.3%   482 2888s
H 6253  6336                     835.2264572  557.65288  33.2%   482 2889s
H 6300  6336                     835.2264569  557.65288  33.2%   482 2889s
  6413  6617  764.38060  530  426  835.22646  557.65288  33.2%   483 2949s
  6696  6905  772.27377  555  439  835.22646  557.65288  33.2%   484 3012s
  6985  7180  781.74505  583  417  835.22646  557.65288  33.2%   485 3086s
  7262  7490  792.66759  607  457  835.22646  557.65288  33.2%   485 3155s
H 7577  7812                     835.2264500  557.65288  33.2%   486 3220s
H 7769  7812                     835.2264498  557.65288  33.2%   485 3220s
H 7900  7826                     835.2264469  557.65288  33.2%   487 3309s
H 7901  7825                     828.0590877  557.65288  32.7%   487 3309s
H 7904  7825                     827.9197767  557.65288  32.6%   487 3309s
H 7905  7784                     817.4526681  557.65288  31.8%   487 3309s
  7914  8108     cutoff  670       817.45267  557.65288  31.8%   488 3376s
  8262  8436  557.65288   20  665  817.45267  557.65288  31.8%   487 3447s
  8594  8786  561.07805   36  634  817.45267  557.65288  31.8%   489 3515s
H 8947  8921                     814.5448951  557.65288  31.5%   489 3619s
H 8950  8910                     810.1488935  557.65288  31.2%   489 3619s
  9093  9287  567.21269   52  757  810.14889  557.65288  31.2%   488 3698s
  9472  9677  569.67352   70  672  810.14889  557.65288  31.2%   487 3767s
  9865  9678  601.55487  146  436  810.14889  557.65288  31.2%   480 3847s
H 9866  9195                     810.0475997  557.65288  31.2%   480 3867s
H 9866  8735                     808.0618269  557.65288  31.0%   480 3870s
H 9866  8298                     807.9134055  557.65288  31.0%   480 3871s
H 9866  7883                     807.3964841  557.65288  30.9%   480 3883s
  9867  7883  571.82193  189  410  807.39648  557.65288  30.9%   480 3885s
H 9867  7489                     806.8926867  557.65288  30.9%   480 3900s
H 9867  7114                     804.6922578  557.65288  30.7%   480 3900s
H 9867  6759                     804.5438363  557.65288  30.7%   480 3904s
  9869  6761  640.68242  453  552  804.54384  557.65288  30.7%   480 3908s
H 9869  6422                     802.6773796  557.65288  30.5%   480 3912s
  9870  6422  642.21417  416  409  802.67738  557.65288  30.5%   480 3915s
  9871  6423  651.53438  517  483  802.67738  557.65288  30.5%   480 3921s
  9872  6424  557.65288   51  408  802.67738  557.65288  30.5%   480 3932s
  9873  6424  561.62786   74  487  802.67738  557.65288  30.5%   480 3937s
  9874  6425  557.78057   35  592  802.67738  557.65288  30.5%   480 3950s
  9875  6426  560.05144   53  606  802.67738  557.65288  30.5%   480 3977s
  9876  6426  710.01316  578  708  802.67738  557.65288  30.5%   480 3993s
  9877  6427  597.42514  274  651  802.67738  557.65288  30.5%   480 4021s
  9878  6428  656.08960  463  589  802.67738  557.65288  30.5%   480 4031s
  9879  6428  560.45320   49  744  802.67738  557.65288  30.5%   480 4071s
  9880  6429  558.62905   42  796  802.67738  557.65288  30.5%   480 4091s
H 9880  6108                     802.5289582  557.65288  30.5%   480 4116s
H 9880  5802                     802.0605077  557.65288  30.5%   480 4116s
H 9880  5512                     799.3626646  557.65288  30.2%   480 4116s
H 9880  5236                     798.6364075  557.65288  30.2%   480 4116s
H 9881  4975                     798.6174604  557.65288  30.2%   480 4138s
H 9881  4726                     795.8402796  557.65288  29.9%   480 4138s
H 9881  4489                     795.0040474  557.65288  29.9%   480 4138s
  9883  4490  557.65288   32  672  795.00405  557.65288  29.9%   480 4153s
  9884  4491  634.19195  279  669  795.00405  557.65288  29.9%   479 4157s
  9885  4491  557.65288   15  729  795.00405  557.65288  29.9%   479 4161s
  9887  4493  623.88868  501  731  795.00405  557.65288  29.9%   479 4174s
  9888  4493  649.15491  489  454  795.00405  557.65288  29.9%   479 4208s
  9889  4494  561.93681   91  643  795.00405  557.65288  29.9%   479 4231s
  9890  4495  663.97235  634  713  795.00405  557.65288  29.9%   479 4240s
  9891  4495  592.65748  169  717  795.00405  557.65288  29.9%   479 4251s
H 9891  4270                     790.5529943  557.65288  29.5%   479 4288s
  9893  4272  559.40116  126  836  790.55299  557.65288  29.5%   479 4360s
  9894  4272  646.63504  462  741  790.55299  557.65288  29.5%   479 4369s
  9895  4273  600.21123  271  838  790.55299  557.65288  29.5%   479 4385s
  9896  4274  557.65288    3  621  790.55299  557.65288  29.5%   479 4442s
  9897  4274  650.15707  508  689  790.55299  557.65288  29.5%   479 4451s
  9898  4275  698.52164  572  730  790.55299  557.65288  29.5%   479 4459s
  9899  4276  566.90860  107  709  790.55299  557.65288  29.5%   479 4484s
  9900  4276  672.24866  523  492  790.55299  557.65288  29.5%   479 4599s
  9901  4277  612.91622  310  801  790.55299  557.65288  29.5%   479 4647s
H 9901  4062                     790.5529917  557.65288  29.5%   479 4901s
H 9901  3859                     790.1468028  557.65288  29.4%   479 4901s
H 9901  3665                     788.5360106  557.65288  29.3%   479 4901s
  9902  3666  557.65288   36  465  788.53601  557.65288  29.3%   479 4969s
  9903  3666  557.65288   35  746  788.53601  557.65288  29.3%   479 5022s
  9904  3667  650.23227  428 1002  788.53601  558.03502  29.2%   478 5051s
  9905  3668  574.09077  172  820  788.53601  558.05399  29.2%   478 5079s
  9906  3668  615.79836  391  604  788.53601  559.28318  29.1%   478 5214s
  9907  3669  559.34672   12  816  788.53601  559.34672  29.1%   478 5233s
  9908  3670  656.10490  443  882  788.53601  560.18019  29.0%   478 5244s
  9909  3670  786.69259  588  849  788.53601  560.18019  29.0%   478 5260s
  9910  3671  560.48195   35  655  788.53601  560.48195  28.9%   478 5292s
  9911  3672  591.75604  261  986  788.53601  560.48315  28.9%   478 5308s
  9912  3672  598.32709  283  888  788.53601  560.50112  28.9%   478 5342s
  9913  3673  565.53384   99 1011  788.53601  561.53972  28.8%   478 5355s
  9914  3674  602.38364  315  949  788.53601  561.53972  28.8%   478 5370s
  9915  3674  604.24414  311  603  788.53601  561.88201  28.7%   478 5411s
  9916  3675  571.81098  166  684  788.53601  562.16816  28.7%   478 5419s
  9917  3676  736.03867  481  651  788.53601  562.31519  28.7%   478 5424s
  9918  3676  562.31519  122  755  788.53601  562.31519  28.7%   478 5455s
  9919  3677  629.85260  415  485  788.53601  562.31519  28.7%   478 5480s
  9920  3678  600.16294  282  813  788.53601  562.32032  28.7%   478 5497s
  9921  3678  654.37476  127  528  788.53601  563.15926  28.6%   478 5528s
  9922  3679  563.20153   30  679  788.53601  563.20153  28.6%   478 5535s
  9923  3680  564.25144  141  990  788.53601  563.53494  28.5%   478 5545s
  9924  3680  566.48912  101 1030  788.53601  563.54771  28.5%   478 5621s
  9925  3681  599.16469  286  816  788.53601  565.45095  28.3%   477 5659s
  9926  3682  623.81800  262  913  788.53601  565.51157  28.3%   477 5674s
  9927  3682  586.55600  314  892  788.53601  565.58567  28.3%   477 5677s
  9928  3683  597.26570  250  940  788.53601  565.58567  28.3%   477 5686s
H 9928  3498                     787.4591250  565.60474  28.2%   477 5700s
  9929  3499  565.60474   72  851  787.45912  565.60474  28.2%   477 5705s
  9930  3499  586.76473  358  835  787.45912  565.75779  28.2%   477 5747s
  9931  3500  597.46552  387 1058  787.45912  565.78090  28.2%   477 5775s
H 9931  3324                     787.1965152  566.60087  28.0%   477 5800s
  9933  3325  566.65446  113  949  787.19652  566.65446  28.0%   477 5807s
  9934  3326  570.08965  194 1016  787.19652  566.96654  28.0%   477 5874s
  9935  3327  650.49038  421 1023  787.19652  567.11450  28.0%   477 5883s
  9936  3327  619.08234  393  845  787.19652  567.31667  27.9%   477 5892s
  9937  3328  575.13793  139  707  787.19652  567.34408  27.9%   477 5901s
  9938  3329  649.11397  471  770  787.19652  567.65299  27.9%   477 5976s
  9939  3329  568.17260   10  848  787.19652  568.17260  27.8%   477 5987s
H 9939  3161                     786.8974123  568.18322  27.8%   477 6083s
  9940  3162  583.85723  322 1048  786.89741  568.18322  27.8%   477 6089s
H 9940  3003                     786.6348025  568.28772  27.8%   477 6173s
  9941  3004  568.28772   21  722  786.63480  568.28772  27.8%   477 6207s
  9942  3004  568.31154   19 1009  786.63480  568.31154  27.8%   477 6229s
  9943  3005  568.59296   91  995  786.63480  568.59296  27.7%   477 6260s
  9944  3006  578.77981  147 1221  786.63480  568.67760  27.7%   477 6283s
  9945  3006  646.24849  506 1285  786.63480  568.69142  27.7%   477 6298s
  9946  3007  569.23912   42 1052  786.63480  569.23912  27.6%   476 6322s
  9947  3008  569.39384   78 1060  786.63480  569.39384  27.6%   476 6333s
  9948  3008  651.61503  431  924  786.63480  569.51095  27.6%   476 6354s
  9949  3009  569.55441   35 1007  786.63480  569.55441  27.6%   476 6364s
  9950  3010  636.96418  358  841  786.63480  569.55441  27.6%   476 6394s
  9951  3010  609.70070  227  618  786.63480  569.84982  27.6%   476 6426s
  9952  3011  607.54343  329  838  786.63480  569.84982  27.6%   476 6439s
H 9952  2859                     786.6096185  569.84982  27.6%   476 6525s
  9954  2860  569.84982   90  742  786.60962  569.84982  27.6%   476 6537s
  9955  2861  622.23852  429  660  786.60962  569.84982  27.6%   476 6557s
  9956  2862  612.05198  307  816  786.60962  569.84982  27.6%   476 6568s
  9957  2862  781.78960  584  627  786.60962  569.84982  27.6%   476 6588s
  9958  2863  569.98722  168  627  786.60962  569.98722  27.5%   476 6597s
H 9958  2718                     786.3522169  569.98722  27.5%   476 6643s
  9960  2722  571.07340   11  653  786.35222  569.98722  27.5%   568 6780s
  9962  2726  571.41492   12  708  786.35222  570.02243  27.5%   568 6853s
  9966  2731  570.27479   13  747  786.35222  570.17168  27.5%   572 6942s
  9973  2742  570.54898   14  854  786.35222  570.18377  27.5%   573 7166s
  9986  2739  570.86723   15  975  786.35222  570.18377  27.5%   582 7307s
  9987  2748  570.93546   16  900  786.35222  570.18377  27.5%   582 7461s
  9997  2799  571.75387   17  925  786.35222  570.50901  27.4%   599 7604s
H10051  2658                     786.3522167  570.50901  27.4%   606 7695s
H10051  2528                     785.7696584  570.50901  27.4%   606 7695s
H10054  2403                     784.9953685  570.50901  27.3%   607 7695s
H10059  2283                     784.2955894  570.50901  27.3%   608 7695s
 10066  2330  574.90452   21  843  784.29559  570.50901  27.3%   610 7760s
 10116  2359  574.91291   22  842  784.29559  570.50901  27.3%   613 7803s
 10162  2383  575.26429   25  746  784.29559  570.50901  27.3%   614 7842s
 10201  2384  578.66169   27  726  784.29559  570.50901  27.3%   614 7892s
H10204  2271                     782.9775670  570.50901  27.1%   614 7892s
H10208  2163                     782.4083691  570.50901  27.1%   614 7892s
 10215  2179  577.04030   27  755  782.40837  570.50901  27.1%   615 7947s
 10234  2185  578.96243   25  817  782.40837  570.50901  27.1%   616 7993s
H10239  2082                     779.0408829  570.50901  26.8%   616 7993s
 10246  2094  577.13844   28  702  779.04088  570.50901  26.8%   617 8033s
H10247  1998                     778.5861509  570.50901  26.7%   617 8033s
H10252  1905                     777.0961251  570.50901  26.6%   617 8033s
 10260  1924  579.56204   34  663  777.09613  570.50901  26.6%   618 8079s
 10282  1931  579.56134   28  763  777.09613  570.50901  26.6%   619 8124s
H10283  1844                     775.0766182  570.50901  26.4%   619 8124s
 10296  1853  578.66169   28  726  775.07662  570.50901  26.4%   619 8157s
H10298  1771                     774.9531630  570.50901  26.4%   619 8158s
H10299  1692                     774.8195101  570.50901  26.4%   619 8158s
H10305  1615                     772.1979426  570.50901  26.1%   620 8158s
 10310  1639  579.09739   28  803  772.19794  570.50901  26.1%   620 8199s
H10335  1572                     768.8238265  570.50901  25.8%   620 8242s
H10338  1504                     768.6496067  570.50901  25.8%   620 8242s
H10346  1438                     768.3924986  570.50901  25.8%   621 8242s
 10349  1451  579.58185   29  889  768.39250  570.50901  25.8%   621 8281s
 10363  1466  579.02200   29  812  768.39250  570.50901  25.8%   621 8323s
H10368  1404                     768.3475723  570.50901  25.7%   621 8324s
 10383  1447  579.17719   31  773  768.34757  570.50901  25.7%   621 8364s
 10432  1445  579.17719   32  739  768.34757  570.50901  25.7%   621 8419s
 10446  1465  579.54525   31  800  768.34757  570.50901  25.7%   622 8464s
 10471  1477  580.06856   33  710  768.34757  570.50901  25.7%   622 8514s
 10491  1501  579.55885   34  759  768.34757  570.50901  25.7%   622 8564s
 10522  1528  579.17719   33  729  768.34757  570.50901  25.7%   623 8613s
 10559  1574  580.06856   35  669  768.34757  570.50901  25.7%   624 8661s
 10617  1613  582.01075   37  672  768.34757  570.50901  25.7%   625 8713s
 10676  1659  582.15257   38  682  768.34757  570.50901  25.7%   625 8767s
 10741  1651  584.46292   39  766  768.34757  570.50901  25.7%   625 8824s
H10744  1592                     768.3475695  570.50901  25.7%   625 8824s
 10752  1589  579.92815   71  776  768.34757  570.50901  25.7%   625 8825s
H10753  1534                     768.3475559  570.50901  25.7%   625 8825s
 10755  1599  584.48139   40  791  768.34756  570.50901  25.7%   625 8885s
H10822  1541                     768.3475528  570.50901  25.7%   626 8954s
H10824  1490                     768.3475510  570.50901  25.7%   626 8954s
H10827  1442                     768.3475499  570.50901  25.7%   626 8954s
H10829  1398                     768.3475454  570.50901  25.7%   626 8954s
H10833  1354                     768.3475443  570.50901  25.7%   625 8954s
H10836  1312                     768.3475438  570.50901  25.7%   625 8955s
 10837  1355  589.52907   43  766  768.34754  570.50901  25.7%   626 9031s
 10880  1375  586.09721   44  729  768.34754  570.50901  25.7%   626 9096s
H10883  1336                     768.3475422  570.50901  25.7%   626 9096s
 10914  1410  586.26188   46  826  768.34754  570.50901  25.7%   626 9161s
 11001  1452  585.43447   49  757  768.34754  570.50901  25.7%   626 9225s
 11073  1494  585.98606   50  573  768.34754  570.50901  25.7%   626 9299s
 11141  1569  588.64840   52  679  768.34754  570.50901  25.7%   626 9368s
 11239  1578  590.44913   55  519  768.34754  570.50901  25.7%   625 9445s
 11281  1583  586.86571   58  542  768.34754  570.50901  25.7%   625 9528s
 11300  1684  586.89534   59  530  768.34754  570.50901  25.7%   625 9607s
 11407  1753  588.38226   63  704  768.34754  570.50901  25.7%   625 9691s
 11514  1846 infeasible   66       768.34754  570.50901  25.7%   623 9767s
 11645  1859  590.36406   74  748  768.34754  570.50901  25.7%   621 11176s
 11702  1977  592.94014   80  771  768.34754  570.50901  25.7%   620 11271s
 11841  1955  591.49825   84  759  768.34754  570.50901  25.7%   618 11393s
 11866  2095  592.48882   85  606  768.34754  570.50901  25.7%   618 11486s
 12014  2082  592.46817   87  645  768.34754  570.50901  25.7%   616 11604s
 12050  2219  593.30058   91  629  768.34754  570.50901  25.7%   616 11702s
 12195  2361  594.12251   93  610  768.34754  570.50901  25.7%   615 11797s
 12337  2524  595.63849   97  601  768.34754  570.50901  25.7%   614 11897s
 12501  2580  596.25543  100  665  768.34754  570.50901  25.7%   612 12010s
 12557  2751  601.05430  102  546  768.34754  570.50901  25.7%   611 12123s
 12728  2931  602.30255  110  714  768.34754  570.50901  25.7%   610 12236s
 12910  2945  600.27214  110  601  768.34754  570.50901  25.7%   607 12387s
 12924  2991  600.45815  111  601  768.34754  570.50901  25.7%   607 12540s
 12970  3097  605.47816  113  582  768.34754  570.50901  25.7%   606 12695s
 13076  3285  603.62847  117  579  768.34754  570.50901  25.7%   605 12830s
 13264  3528  602.94838  131  613  768.34754  570.50901  25.7%   602 12967s
 13510  3856  607.00018  140  695  768.34754  570.50901  25.7%   598 13109s
 13851  4236  608.23480  162  645  768.34754  570.50901  25.7%   592 13242s
 14244  4471  609.86041  188  641  768.34754  570.50901  25.7%   586 13413s
 14495  4868  625.77513  190  617  768.34754  570.50901  25.7%   583 13535s
 14896  5192  613.34821  220  661  768.34754  570.50901  25.7%   577 13684s
 15224  5593  632.77503  228  729  768.34754  570.50901  25.7%   572 13817s
 15628  5704  623.25888  251  576  768.34754  570.50901  25.7%   567 14020s
 15740  5740  623.71431  259  608  768.34754  570.50901  25.7%   566 14229s
 15776  5854  623.73420  260  590  768.34754  570.50901  25.7%   565 14422s
 15912  6215  623.88300  263  613  768.34754  570.50901  25.7%   563 14585s
 16281  6614  625.94502  289  616  768.34754  570.54898  25.7%   560 14728s
 16682  6997  627.95573  318  657  768.34754  570.54898  25.7%   557 14910s
 17085  7283  628.98728  348  644  768.34754  570.54898  25.7%   557 15169s
 17397  7642  635.59891  359  590  768.34754  570.54898  25.7%   557 15337s
 17761  8019  638.60540  376  647  768.34754  570.54898  25.7%   558 15490s

using 500 data points

Set parameter Username
Set parameter LicenseID to value 2694539
Academic license - for non-commercial use only - expires 2026-08-08
Set parameter TimeLimit to value 20
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
TimeLimit  20

Optimize a model with 6288050 rows, 6312550 columns and 37575050 nonzeros
Model fingerprint: 0x13f2b056
Model has 12500 simple general constraints
  12500 INDICATOR
Variable types: 25000 continuous, 6287550 integer (6287550 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+04]
  Objective range  [2e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+04]
  GenCon rhs range [3e+04, 3e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 290100 rows and 862100 columns (presolve time = 5s)...
Presolve removed 290100 rows and 862100 columns (presolve time = 10s)...
Presolve removed 315100 rows and 887100 columns (presolve time = 15s)...
Presolve removed 302600 rows and 874600 columns
Presolve time: 19.72s

Explored 0 nodes (0 simplex iterations) in 20.07 seconds (60.18 work units)
Thread count was 1 (of 14 available processors)

Solution count 0

Time limit reached
Best objective -, best bound -, gap -

No feasible solution found within time limit.

================  SOLUTION  ================

Set parameter Username
Set parameter LicenseID to value 2694539
Academic license - for non-commercial use only - expires 2026-08-08
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G84)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 609230 rows, 615030 columns and 3618030 nonzeros
Model fingerprint: 0xde004b4f
Model has 3000 simple general constraints
  3000 INDICATOR
Variable types: 6000 continuous, 609030 integer (609030 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+04]
  Objective range  [2e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+04]
  GenCon rhs range [3e+04, 3e+04]
  GenCon coe range [1e+00, 1e+00]
Presolve removed 114885 rows and 86385 columns
Presolve time: 4.84s
Presolved: 494345 rows, 528645 columns, 3036765 nonzeros
Variable types: 3000 continuous, 525645 integer (525645 binary)
Found heuristic solution: objective 7607.0228755
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.02s

Barrier performed 0 iterations in 7.02 seconds (17.45 work units)
Barrier solve interrupted - model solved by another algorithm

Concurrent spin time: 0.07s

Solved with dual simplex

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
     601    4.7272722e+02   0.000000e+00   0.000000e+00      7s

Use crossover to convert LP symmetric solution to basic solution...

Root crossover log...

       0 DPushes remaining with DInf 0.0000000e+00                 7s

    1553 PPushes remaining with PInf 0.0000000e+00                 7s
       0 PPushes remaining with PInf 0.0000000e+00                 7s

  Push phase complete: Pinf 0.0000000e+00, Dinf 1.6954681e+01      7s


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4847    4.7272722e+02   0.000000e+00   1.695468e+01      7s
    5907    4.7272721e+02   0.000000e+00   6.804125e+01     10s
    6735    4.7272722e+02   0.000000e+00   0.000000e+00     12s

Root relaxation: objective 4.727272e+02, 6735 iterations, 5.12 seconds (20.42 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  472.72722    0  409 7607.02288  472.72722  93.8%     -   12s
H    0     0                    1897.9750816  472.72722  75.1%     -   13s
H    0     0                    1592.2043408  472.72722  70.3%     -   13s
H    0     0                    1577.2191919  472.72722  70.0%     -   13s
H    0     0                    1576.3596027  472.72722  70.0%     -   13s
H    0     0                    1554.6925703  472.72722  69.6%     -   13s
H    0     0                    1530.2674144  472.72722  69.1%     -   13s
H    0     0                    1522.7933435  472.72722  69.0%     -   13s
H    0     0                    1511.3380126  472.72722  68.7%     -   13s
H    0     0                    1502.0341934  472.72722  68.5%     -   13s
H    0     0                    1482.6100635  472.72722  68.1%     -   13s
H    0     0                    1465.8004952  472.72722  67.7%     -   13s
H    0     0                    1457.4110076  472.72722  67.6%     -   13s
H    0     0                    1455.5214626  472.72722  67.5%     -   13s
H    0     0                    1451.7887254  472.72722  67.4%     -   13s
H    0     0                    1435.2142266  472.72722  67.1%     -   15s
H    0     0                    1409.2497682  472.72722  66.5%     -   16s
H    0     0                    1393.9449956  472.72722  66.1%     -   16s
H    0     0                    1382.0841488  472.72722  65.8%     -   17s
H    0     0                    1357.1795228  472.72722  65.2%     -   18s
H    0     0                    1355.3770218  472.72722  65.1%     -   19s
H    0     0                    1347.8664970  472.72722  64.9%     -   19s
H    0     0                    1346.2466036  472.72722  64.9%     -   19s
H    0     0                    1339.6889334  472.72722  64.7%     -   19s
H    0     0                    1337.8225114  472.72722  64.7%     -   19s
H    0     0                    1303.8025829  472.72722  63.7%     -   20s
H    0     0                    1293.7095815  472.72722  63.5%     -   21s
H    0     0                    1287.0059584  472.72722  63.3%     -   21s
H    0     0                    1267.6308926  472.72722  62.7%     -   21s
     0     0  472.72722    0  464 1267.63089  472.72722  62.7%     -   25s
H    0     0                    1265.3579199  472.72722  62.6%     -   27s
H    0     0                    1258.3665815  472.72722  62.4%     -   32s
H    0     0                    1218.7850162  472.72722  61.2%     -   34s
H    0     0                    1206.4777854  472.72722  60.8%     -   35s
H    0     0                    1175.3014576  472.72722  59.8%     -   36s
H    0     0                    1163.0265719  472.72722  59.4%     -   36s
H    0     0                    1158.2525523  472.72722  59.2%     -   36s
H    0     0                    1154.5982078  472.72722  59.1%     -   38s
H    0     0                    1153.9923116  472.72722  59.0%     -   39s
H    0     0                    1152.2530096  472.72722  59.0%     -   39s
H    0     0                    1152.2330937  472.72722  59.0%     -   39s
H    0     0                    1152.1965208  472.72722  59.0%     -   39s
H    0     0                    1150.3689214  472.72722  58.9%     -   39s
     0     0  472.72722    0  519 1150.36892  472.72722  58.9%     -   40s
     0     0  472.72722    0  409 1150.36892  472.72722  58.9%     -   44s
     0     0  472.72722    0  455 1150.36892  472.72722  58.9%     -   46s
     0     0  472.72722    0  418 1150.36892  472.72722  58.9%     -   51s
     0     0  472.72722    0  463 1150.36892  472.72722  58.9%     -   52s
     0     0  472.72722    0  410 1150.36892  472.72722  58.9%     -   57s
     0     0  472.72722    0  436 1150.36892  472.72722  58.9%     -   59s
     0     0  472.81205    0  417 1150.36892  472.81205  58.9%     -   65s
     0     0  473.69099    0  451 1150.36892  473.69099  58.8%     -   72s
     0     0  482.91783    0  440 1150.36892  482.91783  58.0%     -   75s
     0     0  483.30814    0  434 1150.36892  483.30814  58.0%     -   77s
     0     0  483.30814    0  452 1150.36892  483.30814  58.0%     -   81s
     0     0  488.16095    0  583 1150.36892  488.16095  57.6%     -   88s
     0     0  493.98380    0  562 1150.36892  493.98380  57.1%     -   92s
     0     0  493.99334    0  527 1150.36892  493.99334  57.1%     -   94s
     0     0  493.99334    0  546 1150.36892  493.99334  57.1%     -   96s
     0     0  513.84822    0  551 1150.36892  513.84822  55.3%     -  106s
     0     0  514.38596    0  554 1150.36892  514.38596  55.3%     -  112s
     0     0  514.38596    0  547 1150.36892  514.38596  55.3%     -  115s
     0     0  522.33637    0  610 1150.36892  522.33637  54.6%     -  126s
H    0     0                    1144.4466874  522.33637  54.4%     -  130s
H    0     0                    1139.6718133  522.33637  54.2%     -  132s
H    0     0                    1134.7364004  522.33637  54.0%     -  133s
H    0     0                    1134.6506381  522.33637  54.0%     -  134s
H    0     0                    1120.7359184  522.33637  53.4%     -  136s
H    0     0                    1119.2766278  522.33637  53.3%     -  136s
H    0     0                    1105.9635761  522.33637  52.8%     -  142s
H    0     0                    1090.5372000  522.33637  52.1%     -  143s
H    0     0                    1086.2904881  522.33637  51.9%     -  148s
H    0     0                    1072.6540375  522.33637  51.3%     -  154s
H    0     0                    1058.1429049  522.33637  50.6%     -  154s
H    0     0                    1057.1962151  522.33637  50.6%     -  156s
     0     0  522.48033    0  575 1057.19622  522.48033  50.6%     -  164s
     0     0  522.48819    0  533 1057.19622  522.48819  50.6%     -  167s
     0     0  527.68903    0  569 1057.19622  527.68903  50.1%     -  174s
H    0     0                    1056.5522352  527.68903  50.1%     -  176s
H    0     0                    1056.4749691  527.68903  50.1%     -  176s
H    0     0                    1054.9697841  527.68903  50.0%     -  176s
     0     0  527.70962    0  529 1054.96978  527.70962  50.0%     -  177s
     0     0  527.70962    0  543 1054.96978  527.70962  50.0%     -  178s
     0     0  531.25857    0  596 1054.96978  531.25857  49.6%     -  188s
     0     0  531.80190    0  573 1054.96978  531.80190  49.6%     -  192s
     0     0  531.80190    0  542 1054.96978  531.80190  49.6%     -  193s
     0     0  532.73546    0  532 1054.96978  532.73546  49.5%     -  212s
H    0     0                    1053.5022809  532.73546  49.4%     -  213s
     0     0  532.77288    0  477 1053.50228  532.77288  49.4%     -  218s
     0     0  533.14763    0  471 1053.50228  533.14763  49.4%     -  221s
     0     0  533.14763    0  459 1053.50228  533.14763  49.4%     -  222s
     0     0  534.73245    0  547 1053.50228  534.73245  49.2%     -  232s
     0     0  534.73245    0  513 1053.50228  534.73245  49.2%     -  235s
     0     0  535.07113    0  484 1053.50228  535.07113  49.2%     -  240s
H    0     0                    1051.5849845  535.07113  49.1%     -  249s
H    0     0                    1012.7567406  535.07113  47.2%     -  250s
H    0     0                    1012.7423769  535.07113  47.2%     -  251s
H    0     0                    1008.6382857  535.07113  47.0%     -  253s
H    0     0                    1002.7430946  535.07113  46.6%     -  254s
H    0     0                     990.6857533  535.07113  46.0%     -  255s
H    0     0                     990.2647382  535.07113  46.0%     -  259s
H    0     0                     982.5189526  535.07113  45.5%     -  261s
H    0     0                     981.8056150  535.07113  45.5%     -  263s
H    0     0                     977.5540031  535.07113  45.3%     -  273s
H    0     0                     972.2501258  535.07113  45.0%     -  274s
H    0     0                     963.9545942  535.07113  44.5%     -  289s
H    0     0                     958.8916078  535.07113  44.2%     -  296s
H    0     0                     958.6517903  535.07113  44.2%     -  296s
H    0     0                     955.2250132  535.07113  44.0%     -  298s
H    0     0                     954.2769447  535.07113  43.9%     -  298s
H    0     0                     953.3819264  535.07113  43.9%     -  298s
H    0     0                     950.4827601  535.07113  43.7%     -  302s
H    0     0                     949.6867425  535.07113  43.7%     -  318s
H    0     0                     948.7267821  535.07113  43.6%     -  318s
H    0     0                     947.3114772  535.07113  43.5%     -  322s
H    0     0                     942.8670866  535.07113  43.3%     -  326s
H    0     0                     942.2235176  535.07113  43.2%     -  326s
H    0     0                     941.5989857  535.07113  43.2%     -  326s
H    0     0                     938.7892825  535.07113  43.0%     -  326s
     0     0  535.09820    0  515  938.78928  535.09820  43.0%     -  332s
     0     0  535.09820    0  498  938.78928  535.09820  43.0%     -  333s
     0     0  536.04325    0  587  938.78928  536.04325  42.9%     -  340s
     0     0  536.14553    0  560  938.78928  536.14553  42.9%     -  346s
     0     0  536.15587    0  518  938.78928  536.15587  42.9%     -  347s
     0     0  538.11440    0  577  938.78928  538.11440  42.7%     -  362s
     0     0  538.11440    0  524  938.78928  538.11440  42.7%     -  365s
     0     0  540.17049    0  538  938.78928  540.17049  42.5%     -  372s
     0     0  540.17049    0  583  938.78928  540.17049  42.5%     -  376s
     0     0  542.33070    0  566  938.78928  542.33070  42.2%     -  381s
     0     0  542.46754    0  496  938.78928  542.46754  42.2%     -  383s
     0     0  542.46754    0  492  938.78928  542.46754  42.2%     -  383s
     0     0  543.75470    0  589  938.78928  543.75470  42.1%     -  390s
     0     0  543.76677    0  664  938.78928  543.76677  42.1%     -  444s
     0     0  544.68089    0  537  938.78928  544.68089  42.0%     -  477s
     0     0  545.80957    0  628  938.78928  545.80957  41.9%     -  556s
     0     0  545.80957    0  603  938.78928  545.80957  41.9%     -  558s
     0     0  546.51175    0  541  938.78928  546.51175  41.8%     -  564s
H    0     0                     938.3404307  546.51175  41.8%     -  567s
H    0     0                     937.4381649  546.51175  41.7%     -  568s
H    0     0                     937.0962898  546.51175  41.7%     -  569s
H    0     0                     936.6421288  546.51175  41.7%     -  571s
H    0     0                     931.0698447  546.51175  41.3%     -  571s
     0     0  546.51175    0  516  931.06984  546.51175  41.3%     -  572s
     0     0  547.52076    0  617  931.06984  547.52076  41.2%     -  577s
     0     0  547.78486    0  610  931.06984  547.78486  41.2%     -  640s
     0     0  547.78486    0  552  931.06984  547.78486  41.2%     -  669s
     0     0  548.84164    0  659  931.06984  548.84164  41.1%     -  675s
H    0     0                     929.7512092  548.84164  41.0%     -  678s
H    0     0                     927.1504775  548.84164  40.8%     -  679s
H    0     0                     926.0271353  548.84164  40.7%     -  680s
H    0     0                     920.1754194  548.84164  40.4%     -  680s
H    0     0                     920.0263756  548.84164  40.3%     -  683s
H    0     0                     919.1514045  548.84164  40.3%     - 1388s
H    0     0                     916.0840961  548.84164  40.1%     - 1388s
     0     0  548.94024    0  619  916.08410  548.94024  40.1%     - 1396s
     0     0  548.94024    0  629  916.08410  548.94024  40.1%     - 1398s
     0     0  549.23270    0  546  916.08410  549.23270  40.0%     - 1403s
     0     0  549.29916    0  529  916.08410  549.29916  40.0%     - 1407s
     0     0  549.29916    0  531  916.08410  549.29916  40.0%     - 1409s
     0     0  549.98727    0  564  916.08410  549.98727  40.0%     - 2284s
H    0     0                     915.7319120  549.98727  39.9%     - 2287s
     0     0  550.18626    0  639  915.73191  550.18626  39.9%     - 2294s
     0     0  550.18626    0  584  915.73191  550.18626  39.9%     - 2294s
     0     0  551.16503    0  618  915.73191  551.16503  39.8%     - 2300s
     0     0  551.17414    0  646  915.73191  551.17414  39.8%     - 2305s
     0     0  551.46158    0  703  915.73191  551.46158  39.8%     - 2309s
H    0     0                     914.8925887  551.46158  39.7%     - 2315s
H    0     0                     910.9807256  551.46158  39.5%     - 2315s
     0     0  551.53367    0  542  910.98073  551.53367  39.5%     - 2318s
     0     0  551.53367    0  521  910.98073  551.53367  39.5%     - 2319s
     0     0  551.64605    0  522  910.98073  551.64605  39.4%     - 2338s
     0     0  551.64605    0  540  910.98073  551.64605  39.4%     - 2340s
     0     0  551.65686    0  698  910.98073  551.65686  39.4%     - 2344s
     0     0  551.65686    0  436  910.98073  551.65686  39.4%     - 2352s
H    0     0                     910.7820094  557.65288  38.8%     - 3913s
H    0     0                     909.9103935  557.65288  38.7%     - 3920s
H    0     0                     909.4770968  557.65288  38.7%     - 3929s
H    0     0                     903.9496210  557.65288  38.3%     - 3933s
H    0     0                     902.6753266  557.65288  38.2%     - 3952s
H    0     0                     900.9299109  557.65288  38.1%     - 3960s
     0     2  557.65288    0  403  900.92991  557.65288  38.1%     - 3970s
     1     3  557.65288    1  512  900.92991  557.65288  38.1%  8154 3976s
     2     6  557.65288    2  622  900.92991  557.65288  38.1%  6050 4775s
     5    12  557.65288    3  647  900.92991  557.65288  38.1%  4959 4788s
    11    22  557.65288    4  619  900.92991  557.65288  38.1%  3449 4863s
    21    36  557.65288    5  588  900.92991  557.65288  38.1%  3324 4888s
    35    86  557.65288    6  586  900.92991  557.65288  38.1%  2604 5121s
    85   114  557.65288   10  576  900.92991  557.65288  38.1%  1319 5140s
   113   137  557.65288   11  636  900.92991  557.65288  38.1%  1126 5447s
   136   157  557.65288   12  632  900.92991  557.65288  38.1%  1007 5465s
   156   187  557.65288   13  640  900.92991  557.65288  38.1%   974 5727s
   186   201  557.65288   16  597  900.92991  557.65288  38.1%   900 5749s
   200   216  557.65288   16  617  900.92991  557.65288  38.1%   891 6016s
H  203   216                     899.8704841  557.65288  38.0%   882 6016s
   215   243  557.65288   17  560  899.87048  557.65288  38.0%   863 6035s
   242   264  557.65288   18  561  899.87048  557.65288  38.0%   814 6057s
   264   290  557.65288   19  551  899.87048  557.65288  38.0%   793 6074s
   290   314  557.65288   20  569  899.87048  557.65288  38.0%   771 6098s
   314   337  557.65288   14  599  899.87048  557.65288  38.0%   740 6117s
   337   353  557.65288   22  599  899.87048  557.65288  38.0%   716 6137s
   353   377  557.65288   23  600  899.87048  557.65288  38.0%   712 6159s
   377   402  557.65288   25  582  899.87048  557.65288  38.0%   715 6185s
   402   443  557.65288   26  560  899.87048  557.65288  38.0%   719 6208s
   443   457  557.65288   27  549  899.87048  557.65288  38.0%   688 6232s
   457   488  557.65288   28  565  899.87048  557.65288  38.0%   692 6308s
   488   517  557.65288   29  580  899.87048  557.65288  38.0%   678 6329s
   517   531  557.65288   30  564  899.87048  557.65288  38.0%   669 6352s
H  518   531                     884.5436061  557.65288  37.0%   668 6352s
   531   567  557.65288   31  561  884.54361  557.65288  37.0%   668 6377s
   567   598  557.65288   36  562  884.54361  557.65288  37.0%   655 6404s
   599   646  560.61311   37  593  884.54361  557.65288  37.0%   660 6431s
   647   703  560.61311   38  597  884.54361  557.65288  37.0%   647 6460s
   704   763  561.32918   45  621  884.54361  557.65288  37.0%   632 6491s
   764   818  562.43892   56  584  884.54361  557.65288  37.0%   619 6521s
   819   869  564.95688   63  610  884.54361  557.65288  37.0%   611 6553s
   871   926  570.98563   71  556  884.54361  557.65288  37.0%   609 6587s
H  883   926                     883.9186050  557.65288  36.9%   607 6587s
H  886   926                     883.0755278  557.65288  36.9%   607 6587s
H  892   926                     882.4522124  557.65288  36.8%   607 6587s
   929   995  573.63597   73  518  882.45221  557.65288  36.8%   599 6623s
   998  1009  573.68352   80  509  882.45221  557.65288  36.8%   588 6696s
H 1007  1009                     877.7432976  557.65288  36.5%   587 6696s
  1012  1067  573.68352   81  508  877.74330  557.65288  36.5%   588 6734s
H 1070  1081                     873.9393969  557.65288  36.2%   593 6776s
H 1076  1081                     873.7397790  557.65288  36.2%   593 6776s
H 1077  1081                     868.8251625  557.65288  35.8%   593 6776s
  1084  1146  574.19933   89  542  868.82516  557.65288  35.8%   590 6814s
  1149  1235  575.25453   95  502  868.82516  557.65288  35.8%   589 6853s
  1238  1325  576.65598  103  507  868.82516  557.65288  35.8%   579 6896s
  1329  1428  590.57954  110  574  868.82516  557.65288  35.8%   569 6941s
  1432  1536  591.23971  113  554  868.82516  557.65288  35.8%   558 6985s
  1540  1654  594.69462  120  545  868.82516  557.65288  35.8%   546 7035s
  1662  1787  597.19165  130  465  868.82516  557.65288  35.8%   532 7083s
  1796  1924  598.34306  138  402  868.82516  557.65288  35.8%   516 7135s
  1934  2029  603.60376  155  467  868.82516  557.65288  35.8%   505 7184s
H 2039  2043                     866.8048822  557.65288  35.7%   504 7248s
H 2040  2043                     866.3405327  557.65288  35.6%   504 7248s
H 2045  2043                     866.3329778  557.65288  35.6%   503 7248s
H 2046  2043                     865.6352001  557.65288  35.6%   503 7248s
H 2047  2043                     864.5268631  557.65288  35.5%   503 7248s
H 2052  2043                     862.2980454  557.65288  35.3%   504 7248s
  2053  2185  611.93410  164  488  862.29805  557.65288  35.3%   505 7303s
  2196  2328  614.00732  172  480  862.29805  557.65288  35.3%   495 7362s
  2342  2468  615.29587  181  469  862.29805  557.65288  35.3%   487 7426s
  2484  2593  616.05380  189  513  862.29805  557.65288  35.3%   483 7529s
  2623  2744  618.06355  205  446  862.29805  557.65288  35.3%   480 7616s
  2775  2887  619.48601  222  471  862.29805  557.65288  35.3%   475 7649s
  2920  3022  641.86838  229  493  862.29805  557.65288  35.3%   473 7683s
  3056  3170  642.60256  241  511  862.29805  557.65288  35.3%   474 7716s
H 3066  3170                     861.3476937  557.65288  35.3%   474 7716s
H 3121  3170                     860.5484760  557.65288  35.2%   471 7716s
  3204  3348  658.41208  251  510  860.54848  557.65288  35.2%   473 7750s
  3382  3492  663.32700  263  496  860.54848  557.65288  35.2%   467 7790s
H 3526  3547                     856.5299929  557.65288  34.9%   471 7829s
  3589  3758  666.33797  279  443  856.52999  557.65288  34.9%   472 7869s
  3805  3985  683.44636  298  469  856.52999  557.65288  34.9%   466 7911s
  4032  4171  686.20531  311  423  856.52999  557.65288  34.9%   462 7952s
H 4218  4181                     852.5774653  557.65288  34.6%   463 8012s
H 4219  4181                     851.1321444  557.65288  34.5%   463 8012s
H 4220  4181                     850.5454900  557.65288  34.4%   464 8012s
H 4226  4172                     847.8023877  557.65288  34.2%   464 8012s
  4232  4362  688.35313  338  448  847.80239  557.65288  34.2%   464 8053s
  4423  4556  691.02379  369  429  847.80239  557.65288  34.2%   465 8098s
  4617  4794  696.08567  387  440  847.80239  557.65288  34.2%   469 8143s
H 4857  4805                     843.7872920  557.65288  33.9%   468 8204s
H 4863  4805                     842.9148339  557.65288  33.8%   468 8204s
H 4865  4804                     841.5037300  557.65288  33.7%   468 8204s
H 4867  4802                     838.2676467  557.65288  33.5%   468 8204s
H 4868  4802                     836.1768088  557.65288  33.3%   468 8204s
  4871  5041  696.97728  419  463  836.17681  557.65288  33.3%   468 8286s
  5111  5281  714.96148  447  366  836.17681  557.65288  33.3%   470 9376s
  5351  5503  725.22869  465  401  836.17681  557.65288  33.3%   472 9476s
  5577  5785  733.77468  468  429  836.17681  557.65288  33.3%   477 9656s
  5859  6062  738.49826  493  439  836.17681  557.65288  33.3%   477 11719s
  6137  6336  745.97987  513  380  836.17681  557.65288  33.3%   480 12821s
H 6173  6336                     835.5472359  557.65288  33.3%   482 12821s
H 6253  6336                     835.2264572  557.65288  33.2%   482 12821s
H 6300  6336                     835.2264569  557.65288  33.2%   482 12821s
  6413  6617  764.38060  530  426  835.22646  557.65288  33.2%   483 14424s
  6696  6905  772.27377  555  439  835.22646  557.65288  33.2%   484 15184s
  6985  7180  781.74505  583  417  835.22646  557.65288  33.2%   485 16349s
  7216  7180  594.65387  179  521  835.22646  557.65288  33.2%   485 16350s
  7262  7490  792.66759  607  457  835.22646  557.65288  33.2%   485 18422s
H 7577  7812                     835.2264500  557.65288  33.2%   486 18503s
H 7769  7812                     835.2264498  557.65288  33.2%   485 18503s
H 7900  7826                     835.2264469  557.65288  33.2%   487 18881s
H 7901  7825                     828.0590877  557.65288  32.7%   487 18881s
H 7904  7825                     827.9197767  557.65288  32.6%   487 18881s
H 7905  7784                     817.4526681  557.65288  31.8%   487 18881s
  7914  8108     cutoff  670       817.45267  557.65288  31.8%   488 20004s
  8262  8436  557.65288   20  665  817.45267  557.65288  31.8%   487 21055s
  8594  8786  561.07805   36  634  817.45267  557.65288  31.8%   489 21815s
H 8947  8921                     814.5448951  557.65288  31.5%   489 22064s
H 8950  8910                     810.1488935  557.65288  31.2%   489 22064s
  9093  9287  567.21269   52  757  810.14889  557.65288  31.2%   488 23021s
  9472  9677  569.67352   70  672  810.14889  557.65288  31.2%   487 25971s
  9865  9678  601.55487  146  436  810.14889  557.65288  31.2%   480 26185s
H 9866  9195                     810.0475997  557.65288  31.2%   480 26207s
H 9866  8735                     809.3822569  557.65288  31.1%   480 26209s
H 9866  8298                     809.2338354  557.65288  31.1%   480 26211s
  9867  8298  571.82193  189  409  809.23384  557.65288  31.1%   480 26232s
H 9867  7883                     809.2338339  557.65288  31.1%   480 26927s
H 9867  7489                     809.2338304  557.65288  31.1%   480 26928s
H 9867  7114                     809.2338269  557.65288  31.1%   480 26928s
H 9867  6759                     809.2338223  557.65288  31.1%   480 27875s
H 9867  6421                     809.2138528  557.65288  31.1%   480 27878s
  9868  6422  567.98332  153  406  809.21385  557.65288  31.1%   480 27880s
H 9868  6100                     807.6987494  557.65288  31.0%   480 27887s
H 9868  5795                     807.4264438  557.65288  30.9%   480 27890s
  9870  5796  642.21417  416  407  807.42644  557.65288  30.9%   480 27902s
  9871  5797  651.53438  517  515  807.42644  557.65288  30.9%   480 28847s
  9872  5798  557.65288   51  418  807.42644  557.65288  30.9%   480 28859s
  9873  5798  561.62786   74  563  807.42644  557.65288  30.9%   480 28868s
  9874  5799  557.78057   35  617  807.42644  557.65288  30.9%   480 28878s
  9875  5800  560.05144   53  716  807.42644  557.65288  30.9%   480 28931s
  9876  5800  710.01316  578  816  807.42644  557.65288  30.9%   480 28945s
  9877  5801  597.42514  274  661  807.42644  557.65288  30.9%   480 28958s
  9878  5802  656.08960  463  607  807.42644  557.65288  30.9%   480 28971s
  9879  5802  560.45320   49  698  807.42644  557.65288  30.9%   480 30015s
  9880  5803  558.62905   42  698  807.42644  557.65288  30.9%   480 30032s
  9881  5804  601.91446  306  779  807.42644  557.65288  30.9%   480 30042s
  9882  5804  612.79470  372  671  807.42644  557.65288  30.9%   480 30468s
  9883  5805  557.65288   32  653  807.42644  557.65288  30.9%   480 30480s
H 9883  5515                     807.3111143  557.65288  30.9%   480 30502s
  9884  5516  634.19195  279  506  807.31111  557.65288  30.9%   479 30509s
H 9884  5240                     806.8542317  557.65288  30.9%   479 30529s
  9885  5240  557.65288   15  624  806.85423  557.65288  30.9%   479 30535s
H 9885  4977                     805.5864205  557.65288  30.8%   479 31563s
  9886  4978  562.04551   77  461  805.58642  557.65288  30.8%   479 31570s
  9887  4979  623.88868  501  790  805.58642  557.65288  30.8%   479 32496s
H 9887  4730                     803.9882828  557.65288  30.6%   479 32530s
  9889  4731  561.93681   91  695  803.98828  557.65288  30.6%   479 33469s
  9890  4732  663.97235  634  672  803.98828  557.65288  30.6%   479 33485s
  9891  4732  592.65748  169  610  803.98828  557.65288  30.6%   479 33510s
H 9891  4495                     801.7469931  557.65288  30.4%   479 34555s
H 9891  4270                     800.4331614  557.65288  30.3%   479 34555s
H 9891  4056                     799.9293639  557.65288  30.3%   479 34555s
H 9891  3852                     799.2031069  557.65288  30.2%   479 34563s
H 9891  3659                     798.3668746  557.65288  30.2%   479 34563s
  9893  3661  559.40116  126  752  798.36687  558.10264  30.1%   479 34578s
  9894  3661  646.63504  462  608  798.36687  558.28216  30.1%   479 35517s
  9895  3662  600.21123  271  640  798.36687  558.28216  30.1%   479 35584s
H 9895  3479                     797.6441440  558.28216  30.0%   479 35637s
H 9895  3304                     797.6189600  558.28216  30.0%   479 35670s
H 9895  3138                     796.7792934  558.28216  29.9%   479 35704s
  9896  3139  558.28216    3  499  796.77929  558.28216  29.9%   479 35715s
  9897  3139  650.15707  508  670  796.77929  558.28216  29.9%   479 35746s
  9898  3140  698.52164  572  531  796.77929  558.53733  29.9%   479 35776s
  9899  3141  566.90860  107  870  796.77929  558.93929  29.9%   479 35793s
  9900  3141  672.24866  523  813  796.77929  558.93929  29.9%   479 35812s
  9901  3142  612.91622  310  540  796.77929  559.79538  29.7%   479 35856s
  9902  3143  559.92975   36  892  796.77929  559.92975  29.7%   479 35868s
  9903  3143  560.00968   35  814  796.77929  560.00968  29.7%   479 35878s
  9904  3144  650.23227  428  787  796.77929  560.22441  29.7%   478 35885s
  9905  3145  574.09077  172  832  796.77929  560.22441  29.7%   478 35914s
  9906  3145  615.79836  391  511  796.77929  560.63191  29.6%   478 35972s
  9907  3146  560.88598   12  799  796.77929  560.88598  29.6%   478 35984s
  9908  3147  656.10490  443  844  796.77929  561.38360  29.5%   478 36012s
  9909  3147  786.69259  588  822  796.77929  561.44721  29.5%   478 36020s
  9910  3148  561.44721   35  812  796.77929  561.44721  29.5%   478 36036s
  9911  3149  591.75604  261  597  796.77929  561.44721  29.5%   478 36075s
  9912  3149  598.32709  283  840  796.77929  561.44721  29.5%   478 36092s
  9913  3150  565.53384   99  546  796.77929  561.84835  29.5%   478 36240s
  9914  3151  602.38364  315  778  796.77929  562.08768  29.5%   478 36255s
  9915  3151  604.24414  311  862  796.77929  562.27386  29.4%   478 36263s
  9916  3152  571.81098  166  833  796.77929  562.28082  29.4%   478 36306s
  9917  3153  736.03867  481  547  796.77929  562.83165  29.4%   478 36344s
  9918  3153  562.83678  122  626  796.77929  562.83678  29.4%   478 36367s
  9919  3154  629.85260  415  518  796.77929  563.05239  29.3%   478 36429s
  9920  3155  600.16294  282  799  796.77929  563.37827  29.3%   478 36450s
  9921  3155  654.37476  127  893  796.77929  563.53700  29.3%   478 38627s
  9922  3156  563.65088   30 1001  796.77929  563.65088  29.3%   478 38640s
  9923  3157  564.25144  141  876  796.77929  563.76956  29.2%   478 38651s
  9924  3157  566.48912  101  924  796.77929  563.82401  29.2%   478 38730s
  9925  3158  599.16469  286  946  796.77929  563.93831  29.2%   477 38738s
  9926  3159  623.81800  262  885  796.77929  563.93884  29.2%   477 38751s
  9927  3159  586.55600  314  694  796.77929  566.18081  28.9%   477 40522s
  9928  3160  597.26570  250 1003  796.77929  566.29779  28.9%   477 40532s
  9929  3161  566.33151   72 1062  796.77929  566.33151  28.9%   477 41453s
H 9929  3002                     794.8604157  566.36133  28.7%   477 41712s
  9930  3002  586.76473  358 1145  794.86042  566.36133  28.7%   477 41720s
  9931  3003  597.46552  387  926  794.86042  566.36567  28.7%   477 43779s
  9932  3004  591.57074  206  831  794.86042  566.84196  28.7%   477 43824s
  9933  3004  566.95969  113  789  794.86042  566.95969  28.7%   477 44846s
  9934  3005  570.08965  194  919  794.86042  566.99539  28.7%   477 44859s
  9935  3006  650.49038  421  812  794.86042  567.00251  28.7%   477 44875s
  9936  3006  619.08234  393  609  794.86042  567.00632  28.7%   477 45254s
  9937  3007  575.13793  139  686  794.86042  567.10774  28.7%   477 45263s
  9938  3008  649.11397  471  865  794.86042  567.27899  28.6%   477 45270s
  9939  3008  567.32784   10  715  794.86042  567.32784  28.6%   477 45280s
  9940  3009  583.85723  322  710  794.86042  567.32784  28.6%   477 45294s
H 9940  2858                     794.8467031  567.32784  28.6%   477 47748s
H 9940  2713                     791.2853364  567.32784  28.3%   477 47760s
  9942  2714  567.64681   19  722  791.28534  567.64681  28.3%   477 47778s
  9943  2715  567.65455   91  923  791.28534  567.65455  28.3%   477 48750s
  9944  2716  578.77981  147  774  791.28534  568.12256  28.2%   477 49898s
  9945  2716  646.24849  506  982  791.28534  568.13809  28.2%   477 49911s
H 9945  2579                     786.5428220  568.17712  27.8%   477 50914s
  9946  2580  568.17712   42  493  786.54282  568.17712  27.8%   476 50951s
  9947  2581  568.22848   78  633  786.54282  568.22848  27.8%   476 50961s
  9948  2581  651.61503  431  739  786.54282  568.33150  27.7%   476 51391s
  9949  2582  568.92365   35  914  786.54282  568.92365  27.7%   476 51399s
  9950  2583  636.96418  358 1086  786.54282  568.96034  27.7%   476 51408s
  9951  2583  609.70070  227 1300  786.54282  568.96427  27.7%   476 51420s
  9952  2584  607.54343  329  999  786.54282  568.96427  27.7%   476 51449s
  9953  2585  627.61117  382 1132  786.54282  569.15271  27.6%   476 51458s
  9954  2585  569.29336   90 1142  786.54282  569.29336  27.6%   476 51492s
  9955  2586  622.23852  429  996  786.54282  569.33120  27.6%   476 51513s
  9956  2587  612.05198  307 1087  786.54282  569.36209  27.6%   476 51525s
  9957  2587  781.78960  584  894  786.54282  569.39331  27.6%   476 52568s
  9958  2588  569.48651  168 1119  786.54282  569.48651  27.6%   476 52582s
  9959  2589  569.62084   28  984  786.54282  569.62084  27.6%   476 54410s
  9960  2589  582.96114  294  988  786.54282  569.62084  27.6%   476 54422s
  9961  2590  570.00808   26  777  786.54282  570.00808  27.5%   476 56126s
  9962  2591  570.09863   76 1012  786.54282  570.09863  27.5%   476 57465s
  9963  2591  570.15523   31 1188  786.54282  570.15523  27.5%   476 57472s
  9964  2592  579.44791  139 1240  786.54282  570.18083  27.5%   476 58736s
  9965  2593  601.55487  146 1029  786.54282  570.39376  27.5%   476 58852s